In [ ]:
# %load torch_LSTM_REG.py
#%% ============================import============================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# #載入模型
# LookBackNum = 12     # 參考前12筆
# ForecastNum = 48     # 一天要預測48筆
# features_columns = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']
# target_column = ['Power(mW)']

# # 載入1~17 所有 location 的 AVG(完整)，但要加上sensor_id
# SourceData = pd.DataFrame()

# for i in range(1,18):
#   s = str(i)
#   if i < 10: 
#     s = '0' + s
#   Avg_DataName = os.getcwd() + f'\\ExampleTrainData(AVG)\\AvgDATA_{s}.csv'
#   Incomplete_Avg_DataName = os.getcwd() + f'\\ExampleTrainData(IncompleteAVG)\\IncompleteAvgDATA_{s}.csv'

#   if os.path.exists(Avg_DataName):
#     Avg_temp_data = pd.read_csv(Avg_DataName, encoding='utf-8')
#     Incomplete_Avg_temp_data = pd.read_csv(Incomplete_Avg_DataName, encoding='utf-8')
#     # 先結合AvgDATA和IncompleteAvgDATA
#     combined_df = pd.concat([Avg_temp_data, Incomplete_Avg_temp_data])

#     combined_df['Serial'] = combined_df['Serial'].astype(str)
#     #為了排序先加一個Timestamp(之後要刪掉)
#     combined_df['Timestamp'] = pd.to_datetime(combined_df['Serial'].str[:12], format='%Y%m%d%H%M')
#     #為了計算是不是每一天都是60筆資料先加一個Date(之後要刪掉)
#     combined_df['Date'] = combined_df['Timestamp'].dt.date  # Extract date only for grouping

#     # 將同一天的資料整理成一組，計算裡面是否有60筆。最後篩選出有60筆資料的天數
#     grouped = combined_df.groupby('Date').size()
#     valid_dates_60 = grouped[grouped == 60].index 
    
#     filtered_df_60 = combined_df[combined_df['Date'].isin(valid_dates_60)]

#     filtered_df_60_sorted = filtered_df_60.sort_values(by='Timestamp').reset_index(drop=True)
    
#     filtered_df_60_sorted['sensor_id'] = i

#     SourceData = pd.concat([SourceData, filtered_df_60_sorted], axis=0)

# SourceData = SourceData.drop(columns=['Timestamp']).drop(columns=['Date']).reset_index(drop=True)
# print(SourceData.describe())
# # 透過 describe 發現 WindSpeed 過於不平均，pressure 幾乎都一樣 
# print(SourceData.tail(n=2))
# print(SourceData.shape[0])
# print(SourceData.count())
# print(F"{len(SourceData)}筆資料")

       WindSpeed(m/s)  Pressure(hpa)  Temperature(°C)   Humidity(%)  \
count    74640.000000   74640.000000     74640.000000  74640.000000   
mean         0.357674    1009.832008        31.858715     63.674942   
std          0.734511       6.968556         9.349371     24.624531   
min          0.000000     958.510000        10.870000     11.670000   
25%          0.000000    1004.890000        24.390000     43.100000   
50%          0.000000    1009.040000        31.330000     63.070000   
75%          0.330000    1014.300000        38.230000     85.270000   
max          7.090000    1676.860000        59.870000    100.010000   

       Sunlight(Lux)     Power(mW)     sensor_id  
count   74640.000000  74640.000000  74640.000000  
mean    27718.988908    328.635969      9.610932  
std     30134.627122    511.232791      5.189971  
min       154.500000      0.000000      1.000000  
25%      6143.980000      9.150000      5.000000  
50%     14199.085000     48.970000     10.000000  
75%

In [ ]:
#載入模型
LookBackNum = 12     # 參考前12筆
ForecastNum = 48     # 一天要預測48筆
features_columns = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']
target_column = ['Power(mW)']

# 載入1~17 所有 location 的 AVG(完整)，但要加上sensor_id
SourceData = pd.DataFrame()

for i in range(1,18):
  s = str(i)
  if i < 10: 
    s = '0' + s
  Avg_DataName = os.getcwd() + f'\\ExampleTrainData(AVG)\\AvgDATA_{s}.csv'
  Incomplete_Avg_DataName = os.getcwd() + f'\\ExampleTrainData(IncompleteAVG)\\IncompleteAvgDATA_{s}.csv'

  if os.path.exists(Avg_DataName):
    Avg_temp_data = pd.read_csv(Avg_DataName, encoding='utf-8')
    Incomplete_Avg_temp_data = pd.read_csv(Incomplete_Avg_DataName, encoding='utf-8')
    # 先結合AvgDATA和IncompleteAvgDATA
    combined_df = pd.concat([Avg_temp_data, Incomplete_Avg_temp_data])

    combined_df['Serial'] = combined_df['Serial'].astype(str)
    #為了排序先加一個Timestamp(之後要刪掉)
    combined_df['Timestamp'] = pd.to_datetime(combined_df['Serial'].str[:12], format='%Y%m%d%H%M')
    #為了計算是不是每一天都是60筆資料先加一個Date(之後要刪掉)
    combined_df['Date'] = combined_df['Timestamp'].dt.date  # Extract date only for grouping

    # 將同一天的資料整理成一組，計算裡面是否有60筆。最後篩選出有60筆資料的天數
    grouped = combined_df.groupby('Date').size()
    valid_dates_60 = grouped[grouped == 60].index 
    
    filtered_df_60 = combined_df[combined_df['Date'].isin(valid_dates_60)]

    filtered_df_60_sorted = filtered_df_60.sort_values(by='Timestamp').reset_index(drop=True)
    
    filtered_df_60_sorted['sensor_id'] = i

    SourceData = pd.concat([SourceData, filtered_df_60_sorted], axis=0)

SourceData = SourceData.drop(columns=['Timestamp']).drop(columns=['Date']).reset_index(drop=True)
print(SourceData.describe())
# 透過 describe 發現 WindSpeed 過於不平均，pressure 幾乎都一樣 
print(SourceData.tail(n=2))
print(SourceData.shape[0])
print(SourceData.count())
print(F"{len(SourceData)}筆資料")

In [37]:
# for i in range(1,18):
data = SourceData[SourceData['sensor_id'] == 1]
data.head()

# #%% ============================load data============================
# # 設定 LSTM 往前看的筆數和預測筆數
# LookBackNum = 12
# ForecastNum = 48
# for j in range(1, 18):
    
#     # 載入訓練資料
#     DataName = os.getcwd() + f'/ExampleTrainData(AVG)/AvgDATA_{j}.csv'
#     SourceData = pd.read_csv(DataName, encoding='utf-8')

#     # 迴歸分析用資料
#     Regression_X_train = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values
#     Regression_y_train = SourceData[['Power(mW)']].values

#     # LSTM 用資料
#     AllOutPut = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values

#     # 正規化
#     LSTM_MinMaxModel = MinMaxScaler().fit(AllOutPut)
#     AllOutPut_MinMax = LSTM_MinMaxModel.transform(AllOutPut)

#     X_train = []
#     y_train = []

#     for i in range(LookBackNum, len(AllOutPut_MinMax)):
#         X_train.append(AllOutPut_MinMax[i - LookBackNum:i, :])
#         y_train.append(AllOutPut_MinMax[i, :])

#     X_train = np.array(X_train)
#     y_train = np.array(y_train)


,Serial,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),sensor_id
0,20240101070001,0.00,1016.73,17.50,86.99,470.83,0.08,1
1,20240101071001,0.00,1016.77,17.57,86.97,659.67,0.14,1
2,20240101072001,0.68,1016.92,17.63,86.18,889.67,0.25,1
3,20240101073001,0.61,1016.99,17.74,85.41,1263.83,0.45,1
4,20240101074001,0.00,1017.09,17.90,85.41,2136.83,1.13,1


In [ ]:
#%% ============================定義 PyTorch Dataset============================
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



In [41]:
#%% ============================建置 LSTM 模型============================
class WeatherLSTM(nn.Module):
    def __init__(self, input_size,  output_size):
        super(WeatherLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, 128, batch_first=True)
        self.lstm2 = nn.LSTM(128,64, batch_first=True)
        # self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        # self.lstm4 = nn.LSTM(hidden_sizes[2], hidden_sizes[3], batch_first=True)
        self.fc = nn.Linear(64, output_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        # x, _ = self.lstm3(x)
        # x, _ = self.lstm4(x)
        x = self.dropout(x[:, -1, :])  # 取最後一個時間步的輸出
        x = self.fc(x)
        return x

In [43]:
#%%============================device============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
#parameters

LookBackNum = 12
ForecastNum = 48
# lambda_l1 = 0.001  # L1 正则化系数
# lambda_l2 = 1e-4  # L2 正则化系数
input_size = 5
output_size = 5

## make dir for saving models
os.makedirs("./models", exist_ok=True)

#============================訓練模型============================

# 訓練
epochs = 30
unique_sensor_ids = SourceData['sensor_id'].unique()
for id in unique_sensor_ids:
    data = SourceData[SourceData['sensor_id'] == id]
    
    # 迴歸分析用資料
    Regression_X_train = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values
    Regression_y_train = SourceData[['Power(mW)']].values

    # LSTM 用資料
    AllOutPut = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values

    # 正規化
    LSTM_MinMaxModel = MinMaxScaler().fit(AllOutPut)
    AllOutPut_MinMax = LSTM_MinMaxModel.transform(AllOutPut)

    X_train = []
    y_train = []

    for i in range(LookBackNum, len(AllOutPut_MinMax)):
        X_train.append(AllOutPut_MinMax[i - LookBackNum:i, :])
        y_train.append(AllOutPut_MinMax[i, :])

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    train_dataset = WeatherDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    # 模型初始化
   
    model = WeatherLSTM(input_size, output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # L2 正则化


    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            # 移動資料到 GPU
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # l1_regularization = sum(torch.sum(torch.abs(param)) for param in model.parameters())
            # loss += lambda_l1 * l1_regularization
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Sensor {id} - Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

    # 保存模型
    str_id = str(id)
    if(id < 10):
        str_id = f"0{id}"
    model_path = f"./models/WeatherLSTM_sensor_{str_id}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"LSTM Model for sensor {id} saved at {model_path}")

Sensor 1 - Epoch 1/30, Loss: 0.01026429239124672
Sensor 1 - Epoch 2/30, Loss: 0.0038986070765730965
Sensor 1 - Epoch 3/30, Loss: 0.0034793692514179147
Sensor 1 - Epoch 4/30, Loss: 0.0031781279906943085
Sensor 1 - Epoch 5/30, Loss: 0.0029822552557923028
Sensor 1 - Epoch 6/30, Loss: 0.002771231516667846
Sensor 1 - Epoch 7/30, Loss: 0.0026091903593424668
Sensor 1 - Epoch 8/30, Loss: 0.0024881167651260947
Sensor 1 - Epoch 9/30, Loss: 0.002417341242456643
Sensor 1 - Epoch 10/30, Loss: 0.0023899074671359425
Sensor 1 - Epoch 11/30, Loss: 0.002384847359355437
Sensor 1 - Epoch 12/30, Loss: 0.002373014354467558
Sensor 1 - Epoch 13/30, Loss: 0.002369655595617114
Sensor 1 - Epoch 14/30, Loss: 0.0023511967312565317
Sensor 1 - Epoch 15/30, Loss: 0.002350449206049184
Sensor 1 - Epoch 16/30, Loss: 0.002339508537659964
Sensor 1 - Epoch 17/30, Loss: 0.002346674942177057
Sensor 1 - Epoch 18/30, Loss: 0.002345121547552655
Sensor 1 - Epoch 19/30, Loss: 0.002333643958442972
Sensor 1 - Epoch 20/30, Loss: 0.0

In [47]:

#%% ============================回歸模型============================
# 迴歸分析用資料
data = SourceData[SourceData['sensor_id'] == 17]
Regression_X_train = data[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values
Regression_y_train = data[['Power(mW)']].values
RegressionModel = LinearRegression()
RegressionModel.fit(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train)
#print error
print("R-squared:", RegressionModel.score(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train))
# 保存回歸模型
joblib.dump(RegressionModel, f'WeatherRegression_{pd.Timestamp.now().strftime("%Y-%m-%dT%H_%M_%SZ")}')
print("Regression Model Saved")

print("Intercept:", RegressionModel.intercept_)
print("Coefficients:", RegressionModel.coef_)
print("R-squared:", RegressionModel.score(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train))


R-squared: 0.9016901927465326
Regression Model Saved
Intercept: [-353.82725982]
Coefficients: [[   0.         2674.12392653   60.4660395    23.80303236 1786.83575474]]
R-squared: 0.9016901927465326


In [55]:
LocationCode = int(EXquestion[count].item())
strLocationCode = str(LocationCode)[-2:]
print(f"LocationCode: {strLocationCode}")

LocationCode: 01


In [ ]:
import torch
import os
import sys
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the regression model
RegressionModel = joblib.load('WeatherRegression_2024-11-27T21_39_36Z')

# Load the test data
DataName = os.getcwd() + r"/ExampleTestData/upload.csv"
SourceData = pd.read_csv(DataName, encoding='utf-8')
target = ['序號']
EXquestion = SourceData[target].values

PredictOutput = []  # For weather parameters predictions
PredictPower = []   # For power predictions
count = 0
while count < len(EXquestion):
    sys.stdout.write(f"\rProcessing count: {count} / {len(EXquestion)}")
    sys.stdout.flush()

    LocationCode = int(EXquestion[count].item())
    strLocationCode = str(LocationCode)[-2:]
    if LocationCode < 10:
        strLocationCode = "0" + str(LocationCode)

    # Load corresponding device data
    DataName = os.getcwd() + f"/ExampleTrainData(IncompleteAVG)/IncompleteAvgDATA_{strLocationCode}.csv"
    SourceData = pd.read_csv(DataName, encoding="utf-8")
    ReferTitle = SourceData[['Serial']].values
    ReferData = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values

    # Prepare input data for LSTM
    inputs = []
    for DaysCount in range(len(ReferTitle)):
        if str(int(ReferTitle[DaysCount].item()))[:8] == str(int(EXquestion[count].item()))[:8]:
            TempData = ReferData[DaysCount].reshape(1, -1)  # Shape: (1, 5)
            TempData = LSTM_MinMaxModel.transform(TempData)
            TempData = TempData.squeeze(0)  # Remove the first dimension
            inputs.append(TempData)

    inputs = torch.tensor(inputs, dtype=torch.float32).to(device)  # Convert inputs to Tensor and move to device

    # Load the specific LSTM model for the device
    model_path = f'./models/WeatherLSTM_sensor_{strLocationCode}.pth'
    model = WeatherLSTM(input_size=5, output_size=5).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device,weights_only=True))
    model.eval()

    # Prediction loop (for 48 time steps)
    with torch.no_grad():
        for i in range(ForecastNum):
            # Add the new prediction as input for the next step
            if i > 0:
                new_prediction = torch.tensor(PredictOutput[i - 1], dtype=torch.float32).unsqueeze(0).to(device)  # Shape: (1, 5)
                inputs = torch.cat((inputs, new_prediction), dim=0)  # Concatenate the new prediction

            # Get the last LookBackNum inputs for prediction
            X_test = inputs[-LookBackNum:]  # Shape: (LookBackNum, 5)
            X_test = X_test.unsqueeze(0)  # Add batch dimension -> Shape: (1, LookBackNum, 5)

            # Make a prediction
            predicted = model(X_test)  # Shape: (1, 5)
            PredictOutput.append(predicted.cpu().numpy().flatten())  # Add the predicted values to the output

            # Predict the power using the regression model
            predicted_power = RegressionModel.predict(predicted.cpu().numpy())
            PredictPower.append(np.round(predicted_power, 2).flatten())

    count += 48  # Skip to the next device by incrementing by 48 (since each prediction set is 48 time steps)

print("Prediction Completed")


# test
DataName = os.getcwd() + r"/ExampleTestData/upload.csv"
sd = pd.read_csv(DataName, encoding='utf-8')
# Compute and print SAE
ground_truth = sd[['答案']].values

test_sae = np.abs(ground_truth - PredictPower).sum()

print(f"Testing SAE: {test_sae}")


Processing count: 9552 / 9600Prediction Completed


KeyError: "None of [Index(['答案'], dtype='object')] are in the [columns]"

In [ ]:
# import sys
# #%% ============================預測============================
# # 載入模型
# import torch

# # 設定裝置
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ##記得要改模型名稱
# # 載入 LSTM 模型
# model = WeatherLSTM(input_size=5, hidden_sizes=[512, 256, 128, 32], output_size=5)
# model.load_state_dict(torch.load('WeatherLSTM_2024-11-27T23_26_54Z.pth',weights_only=True))
# model.to(device)
# model.eval()

# # 載入迴歸模型
# RegressionModel = joblib.load('WeatherRegression_2024-11-27T21_39_36Z')

# # 載入測試資料
# DataName = os.getcwd() + r"/ExampleTestData/upload.csv"
# SourceData = pd.read_csv(DataName, encoding='utf-8')
# target = ['序號']
# EXquestion = SourceData[target].values

# PredictOutput = []  # 存放預測值 (天氣參數)
# PredictPower = []   # 存放預測值 (發電量)

# count = 0
# while count < len(EXquestion):
#     sys.stdout.write(f"\rProcessing count: {count} / {len(EXquestion)}")
#     sys.stdout.flush()

#     LocationCode = int(EXquestion[count].item())
#     strLocationCode = str(LocationCode)[-2:]
#     if LocationCode < 10:
#         strLocationCode = "0" + str(LocationCode)

#     DataName = os.getcwd() + f"/ExampleTrainData(IncompleteAVG)/IncompleteAvgDATA_{strLocationCode}.csv"
#     SourceData = pd.read_csv(DataName, encoding="utf-8")
#     ReferTitle = SourceData[['Serial']].values
#     ReferData = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values

#     inputs = []  # 存放參考資料
#     # 找到相同的一天，把 12 個資料都加進 inputs
#     for DaysCount in range(len(ReferTitle)):
#         if str(int(ReferTitle[DaysCount].item()))[:8] == str(int(EXquestion[count].item()))[:8]:
#             TempData = ReferData[DaysCount].reshape(1, -1)  # Shape: (1, 5)
#             TempData = LSTM_MinMaxModel.transform(TempData)
#             TempData = TempData.squeeze(0)  # 去掉第一维，形状变为 (5,)
#             inputs.append(TempData)

#             # print("TempData shape:", TempData.shape)  # 應為 (1, 5)

#     inputs = torch.tensor(inputs, dtype=torch.float32).to(device)  # 將 inputs 轉成 PyTorch Tensor
#     # 預測
#     with torch.no_grad():  # 禁止梯度计算
#         for i in range(ForecastNum):
#             # 将新的预测值加入参考数据 (用自己的预测值往前看)
#             if i > 0:
#                 new_prediction = torch.tensor(PredictOutput[i - 1], dtype=torch.float32).unsqueeze(0).to(device)  # Shape: (1, 5)
#                 inputs = torch.cat((inputs, new_prediction), dim=0)  # 拼接 -> Shape: (N+1, 5)

#             # 切出新的参考数据 12 笔 (往前看 12 笔)
#             X_test = inputs[-LookBackNum:]  # Shape: (LookBackNum, 5)
#             X_test = X_test.unsqueeze(0)  # Add batch dimension -> Shape: (1, LookBackNum, 5)
#             # print("X_test shape:", X_test.shape)  # 应为 (1, LookBackNum, 5)

#             # 前向传播
#             predicted = model(X_test)  # Shape: (1, 5)
#             PredictOutput.append(predicted.cpu().numpy().flatten())  # 添加预测值

#             # 使用回归模型预测发电量
#             predicted_power = RegressionModel.predict(predicted.cpu().numpy())
#             PredictPower.append(np.round(predicted_power, 2).flatten())


#     count += 48  # 每次預測都要預測 48 個，因此加 48 個會切到下一天

# print("Prediction Completed")



# # test
# DataName = os.getcwd() + r"/ExampleTestData/upload.csv"
# SourceData = pd.read_csv(DataName, encoding='utf-8')
# ground_truth = SourceData[['答案']].values

# # 计算 MSE 和 MAE
# test_mse = mean_squared_error(ground_truth, PredictPower)
# test_mae = mean_absolute_error(ground_truth, PredictPower)
# test_sae = np.abs(ground_truth - PredictPower).sum()
# print(f"Testing MSE: {test_mse:.4f}")
# print(f"Testing MAE: {test_mae:.4f}")
# print(f"Testing SAE: {test_sae}")





Processing count: 3120 / 9600

KeyboardInterrupt: 

In [63]:

# test
DataName = os.getcwd() + r"/ExampleTestData/upload.csv"
SourceData = pd.read_csv(DataName, encoding='utf-8')
ground_truth = SourceData[['答案']].values

# Compute and print SAE
PredictOutput = np.array(PredictOutput)
test_sae = np.abs(ground_truth - PredictPower).sum()
print(f"Testing SAE: {test_sae}")

Testing SAE: 4932306.12


In [ ]:
# %%
#寫預測結果寫成新的CSV檔案
# 將陣列轉換為 DataFrame
df = pd.DataFrame(PredictPower, columns=['答案'])

# 將 DataFrame 寫入 CSV 檔案
df_q = pd.read_csv('upload(no answer).csv')
output_df = pd.concat([df_q['序號'], df], axis=1)
output_df.to_csv('output1.csv', index=False)
print('Output CSV File Saved')